Данные были взяты с Kaggle: https://www.kaggle.com/datasets/tklimonova/grocery-website-data-for-ab-test

Заказчик - крупная сеть продуктовых магазинов. Одна из целей, которую ставит перед собой компания - привлечь больше покупателей к скачиванию мобильного приложения и регистрации в программе лояльности.

Руководителю интересно, сможет ли изменение внешнего вида кнопки для скачивания приложения улучшить количество переходов на страницу для его скачивания.

Чтобы найти ответ, был проведен рандомизированный эксперимент:
- Единицей диверсификации в данном эксперименте является IP-адрес.
- Целевая аудитория - посетители сайта без учетной записи.
- Продолжительность теста - 1 неделя.
- Размер тестовых и контрольных групп - 1/3 и 2/3 всех испытуемых.

# Скачивание файла

Для загрузки данных необходимо загрузить файл `kaggle.json`. Скачать его можно из настроек своего аккаунта Kaggle.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!pip install kaggle

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d tklimonova/grocery-website-data-for-ab-test

 71% 1.00M/1.41M [00:00<00:00, 1.51MB/s]
100% 1.41M/1.41M [00:00<00:00, 1.98MB/s]


# Подготовка данных

In [7]:
import pandas as pd
import numpy as np
import zipfile
import warnings

In [8]:
warnings.filterwarnings('ignore')

In [9]:
with zipfile.ZipFile('/content/grocery-website-data-for-ab-test.zip') as myzip:
  print(myzip.namelist())

['grocerywebsiteabtestdata.csv']


In [10]:
with zipfile.ZipFile('/content/grocery-website-data-for-ab-test.zip') as myzip:
  with myzip.open('grocerywebsiteabtestdata.csv') as myfile:
      data = pd.read_csv(myfile)

In [11]:
data.head()

,RecordID,IP Address,LoggedInFlag,ServerID,VisitPageFlag
0,1,39.13.114.2,1,2,0
1,2,13.3.25.8,1,1,0
2,3,247.8.211.8,1,1,0
3,4,124.8.220.3,0,3,0
4,5,60.10.192.7,0,2,0


In [12]:
data.shape

(184588, 5)

Поле `ServerID` содержит информацию о серверах, на которых хранятся данные о группах. Сервер 1 содержит данные для тестовой группы, а серверы 2 и 3 - для контрольной группы.

In [13]:
data['ServerID'].value_counts(normalize=True)

1    0.333667
3    0.333359
2    0.332974
Name: ServerID, dtype: float64

В данном виде данные отражают информацию о отдельных действиях пользователей, а не о самих пользователях. Для анализа нужно оставить по одному полю на каждый уникальный IP-адрес.

In [14]:
data['IP Address'].nunique()

99516

In [15]:
data = data.groupby(['IP Address', 'LoggedInFlag', 'ServerID'])['VisitPageFlag'].sum().reset_index(name='sum_VisitPageFlag')

data['visitFlag'] = data['sum_VisitPageFlag'].apply(lambda x: 1 if x !=0 else 0)
data

,IP Address,LoggedInFlag,ServerID,sum_VisitPageFlag,visitFlag
0,0.0.108.2,0,1,0,0
1,0.0.109.6,1,1,0,0
2,0.0.111.8,0,3,0,0
3,0.0.160.9,1,2,0,0
4,0.0.163.1,0,2,0,0
...,...,...,...,...,...
99758,99.9.53.7,1,2,0,0
99759,99.9.65.2,0,2,0,0
99760,99.9.79.6,1,2,0,0
99761,99.9.86.3,0,1,1,1


Число строк в получившемся датафрейме больше, чем число уникальных IP-адресов.
Скорее всего, есть пользователи, которые несколько раз пользовались сайтом с одного IP-адреса, но в одних случаях заходили в свой профиль, а в других нет.
Поскольку у них есть профили, я им все присвою значение 1 в поле `LoggedInFlag`.

In [16]:
v = data['IP Address'].value_counts()
data.loc[data['IP Address'].isin(v.index[v.gt(1)]), 'LoggedInFlag'] = 1

Удалим из данных нецелевую аудиторию - посетителей, у которых уже есть профили на сайте.

In [17]:
data = data[data['LoggedInFlag'] != 1]
data

,IP Address,LoggedInFlag,ServerID,sum_VisitPageFlag,visitFlag
0,0.0.108.2,0,1,0,0
2,0.0.111.8,0,3,0,0
4,0.0.163.1,0,2,0,0
7,0.0.181.9,0,1,1,1
11,0.0.20.3,0,1,0,0
...,...,...,...,...,...
99746,99.9.206.2,0,1,0,0
99748,99.9.215.4,0,3,1,1
99759,99.9.65.2,0,2,0,0
99761,99.9.86.3,0,1,1,1


Для удобства создадим новое поле, обозначающее группу, в которой находится пользователь.

In [18]:
data['group'] = data['ServerID'].map({1:'Treatment', 2:'Control', 3:'Control'})
data.drop(['ServerID','sum_VisitPageFlag'],axis=1, inplace=True)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49266 entries, 0 to 99762
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IP Address    49266 non-null  object
 1   LoggedInFlag  49266 non-null  int64 
 2   visitFlag     49266 non-null  int64 
 3   group         49266 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.9+ MB


Разделим группы на отдельные датасеты.

In [20]:
data_control = data[data['group'] == 'Control'].copy()
data_control.reset_index(inplace=True, drop = True)

data_test = data[data['group'] == 'Treatment'].copy()
data_test.reset_index(inplace=True, drop = True)

Посмотрим на их статистику:

In [21]:
data_control.describe().T

,count,mean,std,min,25%,50%,75%,max
LoggedInFlag,32797.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
visitFlag,32797.0,0.185932,0.389057,0.0,0.0,0.0,0.0,1.0


In [22]:
data_test.describe().T

,count,mean,std,min,25%,50%,75%,max
LoggedInFlag,16469.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
visitFlag,16469.0,0.232862,0.422668,0.0,0.0,0.0,0.0,1.0


Сразу видно, что среднее значение по полю `visitFlag` отличается - 18.6% для контрольной группы и 23.3% для тестовой. То есть **пользователи из тестовой группы чаще переходили по ссылке для скачивания приложения**. Является ли эта разница статистически значимой - узнаем с помощью тестов.

# Проведение теста.

Для проведения A/B-тестирования набор данных должен удовлетворять предположениям **нормальности** и **однородности дисперсии**. После этого на нем можно проводить проверку гипотез.

Если нормальность и однородность дисперсии обеспечены, то применяется независимый двухвыборочный t-тест (параметрический тест).

Если нормальность и однородность дисперсии не обеспечиваются, то выполняется тест Mann-Whitney U (непараметрический тест).

Для проверки распределения данных используется тест Шапиро-Уилка. Тест Шапиро-Уилка проверяет нулевую гипотезу о том, что данные взяты из нормального распределения.

- $H_0$ - Данные имеют нормальное распределение.
- $H_1$ - Данные не имеют нормального рамспределения.

In [23]:
from scipy.stats import shapiro

significance_level = 0.05

control_stat, p_control = shapiro(data_control["visitFlag"])

test_stat, p_test = shapiro(data_test["visitFlag"])

print(f'Control statistic: {control_stat}, control p-value: {p_control}.')
if p_control < significance_level:
  print(f'H0 отвергается, так как рассчетное значение < {significance_level}. Предположение о нормальности не выполнено.\n')
else:
  print(f'Расчетное значение > {significance_level}, поэтому нельзя отвергнуть H0. Данные распределены нормально.\n')

print(f'Treatment statistic: {test_stat}, control p-value: {p_test}.')
if p_test < significance_level:
  print(f'H0 отвергается, так как рассчетное значение < {significance_level}. Предположение о нормальности не выполнено.\n')
else:
  print(f'Расчетное значение > {significance_level}, поэтому нельзя отвергнуть H0. Данные распределены нормально.\n')

Control statistic: 0.4737479090690613, control p-value: 0.0.
H0 отвергается, так как рассчетное значение < 0.05. Предположение о нормальности не выполнено.

Treatment statistic: 0.5232828259468079, control p-value: 0.0.
H0 отвергается, так как рассчетное значение < 0.05. Предположение о нормальности не выполнено.



Сравнить дисперсии контрольной и проверочной группы можно с помощью теста Левена. Тест Левена проверяет нулевую гипотезу о том, что все входные выборки принадлежат к совокупностям с равными дисперсиями.

- $H_0$ - дисперсии являются однородными.
- $H_1$ - дисперсии не являются однородными.

In [24]:
from scipy.stats import levene

significance_level = 0.05

levene_stat, p_value = levene(data_control['visitFlag'], data_test['visitFlag'])

print(f'Test statistic: {levene_stat}, p-value: {p_value}.')
if p_value < significance_level:
  print(f'H0 отвергается, так как рассчетное значение < {significance_level}. Дисперсии выборок не являются однородными.')
else:
  print(f'Расчетное значение > {significance_level}, поэтому нельзя отвергнуть H0. Дисперсии выборок однородны.')

Test statistic: 150.45993053777246, p-value: 1.544835443107689e-34.
H0 отвергается, так как рассчетное значение < 0.05. Дисперсии выборок не являются однородными.


Данные не прошли тесты на нормальность и однородность дисперсий, поэтому для проведения теста нужно использовать непараметрический тест Манна-Уитни. U-тест Манна-Уитни - это непараметрическая проверка нулевой гипотезы о том, что распределение, лежащее в основе выборки X, совпадает с распределением, лежащим в основе выборки Y.

- $H_0$ - выборки имеют одинаковое распределение.
- $H_1$ - выборки не имеют одинакового распределения.

In [25]:
from scipy.stats import mannwhitneyu

significance_level = 0.05

mw_stat, p_value = mannwhitneyu(data_control['visitFlag'], data_test['visitFlag'])

print(f'Test statistic: {mw_stat}, p-value: {p_value}.')
if p_value < significance_level:
  print(f'H0 отвергается, так как рассчетное значение < {significance_level}. Выборки не имеют одинакового распределения.')
else:
  print(f'Расчетное значение > {significance_level}, поэтому нельзя отвергнуть H0. Выборки имеют одинаковое распределение.')

Test statistic: 257392630.0, p-value: 1.7294299835522993e-34.
H0 отвергается, так как рассчетное значение < 0.05. Выборки не имеют одинакового распределения.


Тест Манна-Уитни показал, что контрольные и тестовые выборки имеют разные распределения, поэтому разница в среднем значении поля `visitFlag` **имеет статистическую значимость**.

# Заключение.

In [26]:
group_count = data.groupby(['group', 'visitFlag'])['group'].count().reset_index(name='Count')
groupped = pd.crosstab(group_count['group'], group_count['visitFlag'], values=group_count['Count'], aggfunc=np.sum, margins=True)

print('Процентное соотношение пользователей, перешедших по ссылке для скачивания в разных группах:')
groupped.div(groupped['All'], axis=0)*100

Процентное соотношение пользователей, перешедших по ссылке для скачивания в разных группах:


visitFlag,0,1,All
group,,,
Control,81.406836,18.593164,100.0
Treatment,76.713826,23.286174,100.0
All,79.838022,20.161978,100.0


Среди пользователей из контрольной группы, 18.6% перешли по ссылке для скачивания приложения.

Среди пользователей из тестовой группы, 23.3% перешли по ссылке для скачивания приложения.

По результатам теста можно сказать, что эта разница имеет статистическую значимость. **Изменение дизайна кнопки для скачивания приложения действительно увеличило число переходов на страницу скачивания на 4.7 процентных пунктов (25%).**